In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
def comp_outside_good(data,name):
    """pre-processing to calculate outside good shares"""
    shares = data[['Market_ID',name,'PPO']].copy()

    group_shares = shares.groupby('Market_ID').sum()
    group_shares['Outside Good Share'] = 1 - group_shares[name]

    data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')
    return data


def comp_nest_shares(x,name):
    """calculate shares within nest"""
    nest_x = x[['Market_ID',name,'PPO']].copy()
    nest_x['ppo_share'] = nest_x[name] * nest_x['PPO']
    nest_x['hmo_share'] = nest_x[name] * (1 - nest_x['PPO'])
    
    group_shares = nest_x.groupby('Market_ID').sum()
    
    x = pd.merge(x, group_shares[['hmo_share','ppo_share']], right_index=True, left_on = 'Market_ID')
    
    x['nest_size'] =   x['PPO'] * x['ppo_share'] + (1 - x['PPO']) * x['hmo_share']
    x = x.drop(labels=['ppo_share','hmo_share'],axis=1)
    return x


data = pd.read_csv('data.csv')
data = comp_outside_good(data,'Inside Good Share')
data = comp_nest_shares(data,'Inside Good Share')
data['ln(Within Nest Share)'] = np.log( data['Inside Good Share']/data['nest_size'] )

In [3]:
#first estimate using logit
class logit(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

        
    def momcond(self, params):
        #unwrap stuff
        shares = np.array(self.endog).transpose()
        exog = np.array(self.exog)
        instr = np.array(self.instrument)
        
        lshare = np.log(shares[0]) -  np.log(shares[1])
        lshare = lshare.transpose()
       
        lshare_fit = np.matmul(exog,params) #linear equation    
        
        xi = lshare_fit - lshare
        g = instr * xi[:, np.newaxis]
        
        return g

# Question 1 - Estimate the Model

In [4]:
#calculate hausmann insturments
plan_dum = pd.get_dummies(data['Plan_ID'], prefix='plan',drop_first=True)
mkt_dum = pd.get_dummies(data['Market_ID'], prefix='plan',drop_first=True)
#hausman_instr =  pd.concat( [data[['Network Score','Satisfaction Score','PPO']], mkt_dum],axis=1)

hausman_instr = plan_dum #unsure if this is right...

#set up data for logit
y = data[['Inside Good Share','Outside Good Share']]

# add ln(inside good share) as regressor like formula
x_nested = data[['Network Score','Satisfaction Score','PPO','Premium','ln(Within Nest Share)']]

I use dummy variables for the plan as my instrument. When regressing these dummy variables on plan, as a result my model has the form 

$$p_{jm} = p_j + \epsilon_{jm}$$ 

as is standard where $\epsilon_{jm}$ is a market specific shock. I estimated a model with the other exogeneous characteristics (i.e. Network score, Satisfaction, PPO) in the instrument. However, the model coefficients where similar, so for simplicitiy I use the dummy variables.


Additionally, I agrue these dummies are also valid instruments for nest share, as 

$$ln(s_{jm|g}) = ln(s_{jm|g}) + \epsilon_{jm}$$

It is plausible that average nested share accross markets will be exogenous from the total share. 

In [5]:
#set up and run model
beta_nested = np.full(len(x_nested.columns),1)
model = logit(y , x_nested, hausman_instr)
result = model.fit(beta_nested, maxiter=2, optim_method='nm', wargs=dict(centered=False))

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.000006
         Iterations: 571
         Function evaluations: 928
Optimization terminated successfully.
         Current function value: 0.005680
         Iterations: 235
         Function evaluations: 380
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        18.74
Model:                                                   logit   Prob (Hansen J):                0.0437
Method:                                                    GMM                                         
Date:                                         Sat, 13 Oct 2018                                         
Time:                                                 15:18:49                                         
No. Observations:                                         3300                                  

In [6]:
#compute market shares

def comp_shares(x,beta,sigma):
    x = x.copy()
    characs = np.array(x[['Network Score','Satisfaction Score','PPO','Premium']])
    x['exp_delta'] = np.exp(np.matmul(characs,beta)/(1-sigma))
    
    #compute Dg = sum_j|g exp(delta_j)
    shares = x[['Market_ID','exp_delta','PPO']].copy()
    shares['PPO_delta'] = x['exp_delta'] * x['PPO']
    shares['HMO_delta'] = x['exp_delta'] * (1 - x['PPO'])
    
    group_shares =  shares.groupby('Market_ID').sum()
    group_shares['PP0_delta_sigma'] = group_shares['PPO_delta']**(sigma)
    group_shares['HMO_delta_sigma'] = group_shares['HMO_delta']**(sigma)
    group_shares['sum_g'] = group_shares['PPO_delta']**(1-sigma) + group_shares['HMO_delta']**(1-sigma) + 1
    #not sure if there should be a 1 at the end of this... (for outside good?)
    x = pd.merge(x,group_shares[['PPO_delta','HMO_delta','PP0_delta_sigma','sum_g']], 
                right_index=True, left_on = 'Market_ID')

    #compute sum_g Dg^(1-sigma)
    x['denom'] = ( (1 - x['PPO'])*x['HMO_delta']**sigma + x['PPO']*x['PPO_delta']**sigma) * (x['sum_g'])
    x['fitted_share'] = x['exp_delta']/x['denom']
    return x[['Market_ID','Plan_ID','PPO','fitted_share']]

In [7]:
#initialize parameters and compute fitted shares
characs = data[['Market_ID','Plan_ID','Network Score','Satisfaction Score','PPO','Premium']]
beta = result.params[:-1]
alpha = abs(beta[3])
sigma = abs(result.params[-1])

fitted_shares = comp_shares(characs,beta,sigma)

In [8]:
#compute total share of each nest
fitted_shares = comp_nest_shares(fitted_shares,'fitted_share')
fitted_shares['nest_shares'] = fitted_shares['fitted_share']/fitted_shares['nest_size']

Characteristics of the fitted shares

In [9]:
print fitted_shares['fitted_share'].mean(), fitted_shares['fitted_share'].max()

0.155548188548 0.40061093912208035


# Question 2 - Compute Elasticities, Markups

## Elasticities

Below are the formulas for the elasticities in the Nested logit

Own Price: $$-\alpha p_{jm} ( \dfrac{1}{1-\sigma} -  \dfrac{1}{1-\sigma}s_{jm|g} -s_{jm} ) $$
Cross Prices Same Nest (good $j$, price $k$): $$\alpha \dfrac{p_{km}}{s_{jm}} s_{km} (\dfrac{\sigma}{1-\sigma}s_{jm|g} + s_{jm} )$$
Cross Prices Different Nest (good $j$, price $k$): $$-\alpha s_{km} p_{km}$$

In [15]:
#aggregate elasticities
fitted_shares['Premium'] = data['Premium']

fitted_shares = fitted_shares.groupby('Plan_ID').mean()

#diagonal formula
nest_shares = np.array(fitted_shares['nest_shares'])
shares = np.array(fitted_shares['fitted_share'])
prices = np.array(fitted_shares['Premium'])
ppo = np.array([fitted_shares['PPO']]) #this one is a matrix

#selector matrices
own_price = np.identity(len(shares))
inside_nest = np.matmul(ppo.transpose(),ppo) + np.matmul((1-ppo.transpose()),(1-ppo)) - own_price
outside_nest = 1 - inside_nest - own_price


#elasticity variables
inside_elasticity = np.matmul( np.array( [ (sigma/(1-sigma) * nest_shares + shares)/shares ] ).transpose(),
          np.array([alpha*prices*shares]))

own_elasticity = -alpha*((1/(1-sigma)) - sigma/(1-sigma) * nest_shares - shares)*prices

outside_elasticity =  shares * alpha * prices

nest_elasticity = own_price*own_elasticity + inside_nest*inside_elasticity + outside_nest*outside_elasticity

Below are the elasticities from the nested logit. Note that the indexing starts at 0

In [16]:
print pd.DataFrame(nest_elasticity)

          0         1         2         3         4         5         6   \
0  -3.328889  0.407455  1.405408  1.119572  0.421476  0.453460  1.518383   
1   0.602507 -4.223542  0.635154  0.505975  1.141629  1.228261  0.686212   
2   1.337650  0.407455 -3.216209  1.123335  0.421476  0.453460  1.523485   
3   1.368524  0.407455  1.442678 -3.483610  0.421476  0.453460  1.558648   
4   0.602507  1.119553  0.635154  0.505975 -4.110862  1.245958  0.686212   
5   0.602507  1.080868  0.635154  0.505975  1.118061 -3.374918  0.686212   
6   1.335640  0.407455  1.408012  1.121647  0.421476  0.453460 -3.792145   
7   0.602507  1.107726  0.635154  0.505975  1.145845  1.232797  0.686212   
8   1.354812  0.407455  1.428223  1.137747  0.421476  0.453460  1.543031   
9   0.602507  1.130796  0.635154  0.505975  1.169708  1.258471  0.686212   
10  0.602507  1.109928  0.635154  0.505975  1.148122  1.235247  0.686212   
11  1.341294  0.407455  1.413972  1.126395  0.421476  0.453460  1.527635   
12  1.400124

## Markups

In [23]:
#solve for marginal costs
def comp_markup(shares,nest_shares):
    shares_vector = np.array([shares])
    
    #set up matrix    
    own_price = np.identity(len(shares))

    #caclulate formula
    own_deriv  =  -alpha*((1/(1-sigma)) - sigma/(1-sigma) * nest_shares - shares)*shares
    
    derivative = own_price *own_deriv
    #take inverse and calc markup
    inv_derivative = np.linalg.inv(derivative)

    markup = - np.matmul(inv_derivative, shares_vector.transpose()) 
    return markup.transpose()[0]



fitted_shares =  comp_shares(characs,beta,sigma)
fitted_shares = comp_nest_shares(fitted_shares,'fitted_share')
fitted_shares['nest_shares'] = fitted_shares['fitted_share']/fitted_shares['nest_size']


nest_shares = np.array(fitted_shares['nest_shares']).squeeze()
shares = np.array(fitted_shares['fitted_share']).squeeze()
prices = np.array(data['Premium']).squeeze()

data['Markup'] = comp_markup(shares,nest_shares)
print data[['Plan_ID','Markup']].groupby('Plan_ID').mean()

           Markup
Plan_ID          
1        0.735307
2        0.662997
3        0.759405
4        0.704811
5        0.678888
6        0.716631
7        0.740775
8        0.732038
9        0.700068
10       0.645045
11       0.683945
12       0.736005
13       0.704012
14       0.703164
15       0.768211
16       0.698950


Overall the markups are higher than with the pure logit model

# Question 3 - Marginal Costs against Plan Characteristics

In [21]:
model_q3 = sm.OLS(data['Premium'] - data['Markup'], data[['Network Score','Satisfaction Score','PPO']])
result_q3 = model_q3.fit()
print result_q3.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 7.914e+05
Date:                Sat, 13 Oct 2018   Prob (F-statistic):               0.00
Time:                        15:49:48   Log-Likelihood:                 3257.8
No. Observations:                3300   AIC:                            -6510.
Df Residuals:                    3297   BIC:                            -6491.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Network Score          2.4226      0